# USA MAP


In [17]:
# Librerías principales para manipulación de datos
import pandas as pd  # Manejo de datos en formato tabular (DataFrames)
import numpy as np  # Funciones matemáticas y operaciones con arrays

# Librerías para construir pipelines y preprocesamiento
from sklearn.pipeline import Pipeline  # Para crear pipelines que incluyan pasos de preprocesamiento y modelo
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder  # Para codificar variables categóricas y escalar variables numéricas
from sklearn.compose import ColumnTransformer  # Para aplicar diferentes transformaciones a diferentes tipos de columnas

# Modelos de machine learning
from sklearn.ensemble import RandomForestClassifier, StackingClassifier  # Algoritmo de clasificación basado en árboles y Stacking
from xgboost import XGBClassifier  # Algoritmo XGBoost, un modelo de boosting eficiente
from catboost import CatBoostClassifier  # Algoritmo CatBoost, un modelo de boosting que maneja variables categóricas internamente

# Librerías para validación cruzada y evaluación de modelos
from sklearn.model_selection import cross_validate, StratifiedKFold  # Validación cruzada y división estratificada de los datos
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score  # Métricas para evaluar el rendimiento de los modelos

# Librerías auxiliares para medir tiempos y guardar modelos
import time  # Para medir el tiempo de ejecución de los modelos
import os  # Para manejar el sistema de archivos, como la creación de directorios
import joblib  # Para guardar y cargar modelos entrenados
from datetime import datetime  # Para obtener la fecha y hora actual

# Configuración cuaderno Jupiter
# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Si también quieres ajustar el ancho de las columnas y ver todo el contenido
pd.set_option('display.max_colwidth', None)

# Si quieres ver todas las filas, puedes usar:
# pd.set_option('display.max_rows', None)

# Restablecer a los valores predeterminados
# pd.reset_option('display.max_columns')
# pd.reset_option('display.max_colwidth')
# pd.reset_option('display.max_rows')



# Paso 1: Cargar los datos (CSV limpio)
data = pd.read_csv('../data/airline_passenger_satisfaction.csv')

# Paso 1.1: Eliminar registros con valores nulos en la columna 'Arrival Delay in Minutes'
data = data.dropna(subset=['Arrival Delay in Minutes'])

# Paso 1.2: Eliminar las columnas 'Unnamed: 0' y 'id' porque no aportan valor al análisis
data = data.drop(['Unnamed: 0', 'id'], axis=1)

# Paso 1.3: Convertir las etiquetas de 'satisfaction' de cadenas a valores numéricos (0 y 1)
data['satisfaction'] = data['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})
y = data['satisfaction']
X = data.drop('satisfaction', axis=1)

# Paso 2: Codificar manualmente las variables categóricas usando `map`
# Definir las columnas categóricas
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# Crear un mapeo para cada columna categórica
categorical_mappings = {
    'Gender': {'Male': 0, 'Female': 1},
    'Customer Type': {'Loyal Customer': 1, 'disloyal Customer': 0},
    'Type of Travel': {'Business travel': 1, 'Personal Travel': 0},
    'Class': {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
}

# Aplicar el mapeo
for col, mapping in categorical_mappings.items():
    X[col] = X[col].map(mapping)

# Las variables ordinales ya tienen valores numéricos entre 0 y 5, por lo que no necesitan codificación adicional
ordinal_cols = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 
                'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 
                'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

# Paso 3: Crear el preprocesador de columnas (solo escalar las numéricas)
numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols)  # Escalado para las columnas numéricas
    ]
)

# Paso 4: Crear los pipelines específicos de cada modelo
pipeline_rf = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_xgb = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss'))
])

pipeline_catboost = Pipeline(steps=[
    ('model', CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=False))
])

# Paso 5: Definir el modelo de Stacking
stacking_model = StackingClassifier(
    estimators=[
        ('catboost', pipeline_catboost),  # CatBoost
        ('xgboost', pipeline_xgb),        # XGBoost
        ('random_forest', pipeline_rf)    # Random Forest
    ],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42),
    cv=5
)

# Paso 6: Definir las métricas y la validación cruzada
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Paso 7: Función para evaluar un modelo
def evaluar_modelo_con_tiempo(pipeline, X, y, cv):
    start_time = time.time()  # Tiempo de inicio
    cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=False)
    end_time = time.time()  # Tiempo final
    elapsed_time = end_time - start_time  # Tiempo total de procesamiento
    return cv_results, elapsed_time

# Paso 8: Definir el KFold para la validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lista de resultados
resultados = []

# Evaluar los modelos base y el modelo de Stacking
modelos = {
    'Random Forest': pipeline_rf,
    'XGBoost': pipeline_xgb,
    'CatBoost': pipeline_catboost,
    'Stacking': stacking_model
}

# Paso 9: Evaluar cada modelo con validación cruzada y almacenar métricas y tiempos
for nombre_modelo, modelo in modelos.items():
    print(f"\nEvaluando {nombre_modelo}...")
    
    # Evaluar el modelo y calcular métricas y tiempo
    cv_resultados, tiempo = evaluar_modelo_con_tiempo(modelo, X, y, kf)
    
    # Almacenar resultados en un diccionario
    resultados.append({
        'modelo': nombre_modelo,
        'accuracy_mean': cv_resultados['test_accuracy'].mean(),
        'precision_mean': cv_resultados['test_precision'].mean(),
        'recall_mean': cv_resultados['test_recall'].mean(),
        'f1_mean': cv_resultados['test_f1'].mean(),
        'roc_auc_mean': cv_resultados['test_roc_auc'].mean(),
        'tiempo_procesamiento': tiempo,
        'modelo_pipeline': modelo  # Guardar el pipeline del modelo
    })

# Convertimos los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)
print("\nResultados finales:")
print(df_resultados)

# Paso 10: Seleccionar el mejor modelo en 'accuracy'
mejor_modelo = df_resultados.loc[df_resultados['accuracy_mean'].idxmax()]

print(f"\nEl mejor modelo es: {mejor_modelo['modelo']} con una accuracy media de: {mejor_modelo['accuracy_mean']}")

# Paso 11: Guardar el mejor modelo en un archivo con el nombre del algoritmo y la fecha/hora
def guardar_modelo_con_nombre(modelo, nombre_modelo):
    os.makedirs('../data/modelos_entrenamiento', exist_ok=True)
    
    # Obtener la fecha y hora actual para el nombre del archivo
    fecha_hora_actual = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Crear el nombre del archivo
    nombre_archivo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.pkl"
    
    # Guardar el modelo
    joblib.dump(modelo, f'../data/modelos_entrenamiento/{nombre_archivo}')
    print(f"Modelo guardado exitosamente con el nombre: {nombre_archivo}")

# Guardar el mejor modelo
guardar_modelo_con_nombre(mejor_modelo['modelo_pipeline'], mejor_modelo['modelo'])



Evaluando Random Forest...

Evaluando XGBoost...

Evaluando CatBoost...

Evaluando Stacking...

Resultados finales:
          modelo  accuracy_mean  precision_mean  recall_mean   f1_mean  roc_auc_mean  tiempo_procesamiento                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [18]:
# Librerías principales para manipulación de datos
import pandas as pd  # Manejo de datos en formato tabular (DataFrames)
import numpy as np  # Funciones matemáticas y operaciones con arrays

# Librerías para construir pipelines y preprocesamiento
from sklearn.pipeline import Pipeline  # Para crear pipelines que incluyan pasos de preprocesamiento y modelo
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder  # Para codificar variables categóricas y escalar variables numéricas
from sklearn.compose import ColumnTransformer  # Para aplicar diferentes transformaciones a diferentes tipos de columnas

# Modelos de machine learning
from sklearn.ensemble import RandomForestClassifier, StackingClassifier  # Algoritmo de clasificación basado en árboles y Stacking
from xgboost import XGBClassifier  # Algoritmo XGBoost, un modelo de boosting eficiente
from catboost import CatBoostClassifier  # Algoritmo CatBoost, un modelo de boosting que maneja variables categóricas internamente

# Librerías para validación cruzada y evaluación de modelos
from sklearn.model_selection import cross_validate, StratifiedKFold  # Validación cruzada y división estratificada de los datos
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score  # Métricas para evaluar el rendimiento de los modelos

# Librerías auxiliares para medir tiempos y guardar modelos
import time  # Para medir el tiempo de ejecución de los modelos
import os  # Para manejar el sistema de archivos, como la creación de directorios
import joblib  # Para guardar y cargar modelos entrenados
from datetime import datetime  # Para obtener la fecha y hora actual

# Configuración cuaderno Jupyter: Mostrar todas las columnas y ajustar ancho de las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Paso 1: Cargar los datos (CSV limpio)
data = pd.read_csv('../data/airline_passenger_satisfaction.csv')

# Paso 1.1: Eliminar registros con valores nulos en la columna 'Arrival Delay in Minutes'
data = data.dropna(subset=['Arrival Delay in Minutes'])

# Paso 1.2: Eliminar las columnas 'Unnamed: 0' y 'id' porque no aportan valor al análisis
data = data.drop(['Unnamed: 0', 'id'], axis=1)

# Paso 1.3: Convertir las etiquetas de 'satisfaction' de cadenas a valores numéricos (0 y 1)
data['satisfaction'] = data['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})
y = data['satisfaction']
X = data.drop('satisfaction', axis=1)

# Paso 2: Codificar manualmente las variables categóricas usando `map`
# Definir las columnas categóricas
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# Crear un mapeo para cada columna categórica
categorical_mappings = {
    'Gender': {'Male': 0, 'Female': 1},
    'Customer Type': {'Loyal Customer': 1, 'disloyal Customer': 0},
    'Type of Travel': {'Business travel': 1, 'Personal Travel': 0},
    'Class': {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
}

# Aplicar el mapeo
for col, mapping in categorical_mappings.items():
    X[col] = X[col].map(mapping)

# Las variables ordinales ya tienen valores numéricos entre 0 y 5, por lo que no necesitan codificación adicional
ordinal_cols = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 
                'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 
                'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

# Paso 3: Crear el preprocesador de columnas (solo escalar las numéricas)
numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols)  # Escalado para las columnas numéricas
    ]
)

# Paso 4: Crear los pipelines específicos de cada modelo
pipeline_rf = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_xgb = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss'))
])

pipeline_catboost = Pipeline(steps=[
    ('model', CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=False))
])

# Paso 5: Definir el modelo de Stacking
stacking_model = StackingClassifier(
    estimators=[
        ('catboost', pipeline_catboost),  # CatBoost
        ('xgboost', pipeline_xgb),        # XGBoost
        ('random_forest', pipeline_rf)    # Random Forest
    ],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42),
    cv=5
)

# Paso 6: Definir las métricas y la validación cruzada
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Paso 7: Función para evaluar un modelo
def evaluar_modelo_con_tiempo(pipeline, X, y, cv):
    start_time = time.time()  # Tiempo de inicio
    cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=False)
    end_time = time.time()  # Tiempo final
    elapsed_time = end_time - start_time  # Tiempo total de procesamiento
    return cv_results, elapsed_time

# Paso 8: Definir el KFold para la validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lista de resultados
resultados = []

# Evaluar los modelos base y el modelo de Stacking
modelos = {
    'Random Forest': pipeline_rf,
    'XGBoost': pipeline_xgb,
    'CatBoost': pipeline_catboost,
    'Stacking': stacking_model
}

# Paso 9: Evaluar cada modelo con validación cruzada y almacenar métricas y tiempos
for nombre_modelo, modelo in modelos.items():
    print(f"\nEvaluando {nombre_modelo}...")
    
    # Evaluar el modelo y calcular métricas y tiempo
    cv_resultados, tiempo = evaluar_modelo_con_tiempo(modelo, X, y, kf)
    
    # Almacenar resultados en un diccionario
    resultados.append({
        'modelo': nombre_modelo,
        'accuracy_mean': cv_resultados['test_accuracy'].mean(),
        'precision_mean': cv_resultados['test_precision'].mean(),
        'recall_mean': cv_resultados['test_recall'].mean(),
        'f1_mean': cv_resultados['test_f1'].mean(),
        'roc_auc_mean': cv_resultados['test_roc_auc'].mean(),
        'tiempo_procesamiento': tiempo,
        'modelo_pipeline': modelo  # Guardar el pipeline del modelo
    })

# Convertimos los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)

# Mostrar todas las columnas del DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

print("\nResultados finales:")
print(df_resultados)

# Paso 10: Seleccionar el mejor modelo en 'accuracy'
mejor_modelo = df_resultados.loc[df_resultados['accuracy_mean'].idxmax()]

print(f"\nEl mejor modelo es: {mejor_modelo['modelo']} con una accuracy media de: {mejor_modelo['accuracy_mean']}")

# Paso 11: Guardar el mejor modelo en un archivo con el nombre del algoritmo y la fecha/hora
def guardar_modelo_con_nombre(modelo, nombre_modelo):
    os.makedirs('data/modelos_entrenamiento', exist_ok=True)
    
    # Obtener la fecha y hora actual para el nombre del archivo
    fecha_hora_actual = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Crear el nombre del archivo
    nombre_archivo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.pkl"
    
    # Guardar el modelo
    joblib.dump(modelo, f'data/modelos_entrenamiento/{nombre_archivo}')
    print(f"Modelo guardado exitosamente con el nombre: {nombre_archivo}")

# Guardar el mejor modelo
guardar_modelo_con_nombre(mejor_modelo['modelo_pipeline'], mejor_modelo['modelo'])



Evaluando Random Forest...

Evaluando XGBoost...

Evaluando CatBoost...

Evaluando Stacking...

Resultados finales:
          modelo  accuracy_mean  precision_mean  recall_mean   f1_mean  roc_auc_mean  tiempo_procesamiento                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [19]:
# Librerías principales para manipulación de datos
import pandas as pd  # Manejo de datos en formato tabular (DataFrames)
import numpy as np  # Funciones matemáticas y operaciones con arrays

# Librerías para construir pipelines y preprocesamiento
from sklearn.pipeline import Pipeline  # Para crear pipelines que incluyan pasos de preprocesamiento y modelo
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder  # Para codificar variables categóricas y escalar variables numéricas
from sklearn.compose import ColumnTransformer  # Para aplicar diferentes transformaciones a diferentes tipos de columnas

# Modelos de machine learning
from sklearn.ensemble import RandomForestClassifier, StackingClassifier  # Algoritmo de clasificación basado en árboles y Stacking
from xgboost import XGBClassifier  # Algoritmo XGBoost, un modelo de boosting eficiente
from catboost import CatBoostClassifier  # Algoritmo CatBoost, un modelo de boosting que maneja variables categóricas internamente

# Librerías para validación cruzada y evaluación de modelos
from sklearn.model_selection import cross_validate, StratifiedKFold  # Validación cruzada y división estratificada de los datos
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score  # Métricas para evaluar el rendimiento de los modelos

# Librerías auxiliares para medir tiempos y guardar modelos
import time  # Para medir el tiempo de ejecución de los modelos
import os  # Para manejar el sistema de archivos, como la creación de directorios
import joblib  # Para guardar y cargar modelos entrenados
from datetime import datetime  # Para obtener la fecha y hora actual

# Configuración cuaderno Jupyter: Mostrar todas las columnas y ajustar ancho de las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Paso 1: Cargar los datos (CSV limpio)
data = pd.read_csv('../data/airline_passenger_satisfaction.csv')

# Paso 1.1: Eliminar registros con valores nulos en la columna 'Arrival Delay in Minutes'
data = data.dropna(subset=['Arrival Delay in Minutes'])

# Paso 1.2: Eliminar las columnas 'Unnamed: 0' y 'id' porque no aportan valor al análisis
data = data.drop(['Unnamed: 0', 'id'], axis=1)

# Paso 1.3: Convertir las etiquetas de 'satisfaction' de cadenas a valores numéricos (0 y 1)
data['satisfaction'] = data['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})
y = data['satisfaction']
X = data.drop('satisfaction', axis=1)

# Paso 2: Codificar manualmente las variables categóricas usando `map`
# Definir las columnas categóricas
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# Crear un mapeo para cada columna categórica
categorical_mappings = {
    'Gender': {'Male': 0, 'Female': 1},
    'Customer Type': {'Loyal Customer': 1, 'disloyal Customer': 0},
    'Type of Travel': {'Business travel': 1, 'Personal Travel': 0},
    'Class': {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
}

# Aplicar el mapeo
for col, mapping in categorical_mappings.items():
    X[col] = X[col].map(mapping)

# Las variables ordinales ya tienen valores numéricos entre 0 y 5, por lo que no necesitan codificación adicional
ordinal_cols = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 
                'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 
                'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

# Paso 3: Crear el preprocesador de columnas (solo escalar las numéricas)
numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols)  # Escalado para las columnas numéricas
    ]
)

# Paso 4: Crear los pipelines específicos de cada modelo
pipeline_rf = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_xgb = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss'))
])

pipeline_catboost = Pipeline(steps=[
    ('model', CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=False))
])

# Paso 5: Definir el modelo de Stacking
stacking_model = StackingClassifier(
    estimators=[
        ('catboost', pipeline_catboost),  # CatBoost
        ('xgboost', pipeline_xgb),        # XGBoost
        ('random_forest', pipeline_rf)    # Random Forest
    ],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42),
    cv=5
)

# Paso 6: Definir las métricas y la validación cruzada
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Paso 7: Función para evaluar un modelo
def evaluar_modelo_con_tiempo(pipeline, X, y, cv):
    start_time = time.time()  # Tiempo de inicio
    cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=False)
    end_time = time.time()  # Tiempo final
    elapsed_time = end_time - start_time  # Tiempo total de procesamiento
    return cv_results, elapsed_time

# Paso 8: Definir el KFold para la validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lista de resultados
resultados = []

# Evaluar los modelos base y el modelo de Stacking
modelos = {
    'Random Forest': pipeline_rf,
    'XGBoost': pipeline_xgb,
    'CatBoost': pipeline_catboost,
    'Stacking': stacking_model
}

# Paso 9: Evaluar cada modelo con validación cruzada y almacenar métricas y tiempos
for nombre_modelo, modelo in modelos.items():
    print(f"\nEvaluando {nombre_modelo}...")
    
    # Evaluar el modelo y calcular métricas y tiempo
    cv_resultados, tiempo = evaluar_modelo_con_tiempo(modelo, X, y, kf)
    
    # Almacenar resultados en un diccionario
    resultados.append({
        'modelo': nombre_modelo,
        'accuracy_mean': cv_resultados['test_accuracy'].mean(),
        'precision_mean': cv_resultados['test_precision'].mean(),
        'recall_mean': cv_resultados['test_recall'].mean(),
        'f1_mean': cv_resultados['test_f1'].mean(),
        'roc_auc_mean': cv_resultados['test_roc_auc'].mean(),
        'tiempo_procesamiento': tiempo,
        'modelo_pipeline': modelo  # Guardar el pipeline del modelo
    })

# Convertimos los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)

# Paso 10: Crear una vista truncada para mostrar en la consola (sin los pipelines)
df_vista_consola = df_resultados.drop(columns=['modelo_pipeline'])

# Mostrar los resultados sin la columna `modelo_pipeline` en la consola
print("\nResultados finales:")
print(df_vista_consola)

# Paso 11: Guardar el DataFrame completo (incluyendo pipelines) en un CSV
output_csv = 'resultados_modelos_completos.csv'
df_resultados.to_csv(output_csv, index=False)
print(f"\nLos resultados completos se han guardado en el archivo: {output_csv}")

# Paso 12: Seleccionar el mejor modelo en 'accuracy'
mejor_modelo = df_resultados.loc[df_resultados['accuracy_mean'].idxmax()]

print(f"\nEl mejor modelo es: {mejor_modelo['modelo']} con una accuracy media de: {mejor_modelo['accuracy_mean']}")

# Paso 13: Guardar el mejor modelo en un archivo con el nombre del algoritmo y la fecha/hora
def guardar_modelo_con_nombre(modelo, nombre_modelo):
    os.makedirs('data/modelos_entrenamiento', exist_ok=True)
    
    # Obtener la fecha y hora actual para el nombre del archivo
    fecha_hora_actual = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Crear el nombre del archivo
    nombre_archivo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.pkl"
    
    # Guardar el modelo
    joblib.dump(modelo, f'data/modelos_entrenamiento/{nombre_archivo}')
    print(f"Modelo guardado exitosamente con el nombre: {nombre_archivo}")

# Guardar el mejor modelo
guardar_modelo_con_nombre(mejor_modelo['modelo_pipeline'], mejor_modelo['modelo'])



Evaluando Random Forest...


KeyboardInterrupt: 

In [14]:
# Librerías principales para manipulación de datos
import pandas as pd  # Manejo de datos en formato tabular (DataFrames)
import numpy as np  # Funciones matemáticas y operaciones con arrays

# Librerías para construir pipelines y preprocesamiento
from sklearn.pipeline import Pipeline  # Para crear pipelines que incluyan pasos de preprocesamiento y modelo
from sklearn.preprocessing import StandardScaler  # Para escalar variables numéricas
from sklearn.compose import ColumnTransformer  # Para aplicar diferentes transformaciones a diferentes tipos de columnas

# Modelos de machine learning
from sklearn.ensemble import RandomForestClassifier, StackingClassifier  # Algoritmo de clasificación basado en árboles y Stacking
from xgboost import XGBClassifier  # Algoritmo XGBoost, un modelo de boosting eficiente
from catboost import CatBoostClassifier  # Algoritmo CatBoost, un modelo de boosting que maneja variables categóricas internamente

# Librerías para validación cruzada y evaluación de modelos
from sklearn.model_selection import cross_validate, StratifiedKFold, train_test_split  # Validación cruzada y división estratificada de los datos
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score  # Métricas para evaluar el rendimiento de los modelos

# Librerías auxiliares para medir tiempos y guardar modelos
import time  # Para medir el tiempo de ejecución de los modelos
import os  # Para manejar el sistema de archivos, como la creación de directorios
import joblib  # Para guardar y cargar modelos entrenados
from datetime import datetime, timedelta  # Para obtener la fecha y hora actual

# Configuración cuaderno Jupyter: Mostrar todas las columnas y ajustar ancho de las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Paso 1: Cargar los datos (CSV limpio)
data = pd.read_csv('../data/airline_passenger_satisfaction.csv')

# Paso 1.1: Eliminar registros con valores nulos en la columna 'Arrival Delay in Minutes'
data = data.dropna(subset=['Arrival Delay in Minutes'])

# Paso 1.2: Eliminar las columnas 'Unnamed: 0' y 'id' porque no aportan valor al análisis
data = data.drop(['Unnamed: 0', 'id'], axis=1)

# Paso 1.3: Convertir las etiquetas de 'satisfaction' de cadenas a valores numéricos (0 y 1)
data['satisfaction'] = data['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})
y = data['satisfaction']
X = data.drop('satisfaction', axis=1)

# Paso 2: Codificar manualmente las variables categóricas usando `map`
# Definir las columnas categóricas
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# Crear un mapeo para cada columna categórica
categorical_mappings = {
    'Gender': {'Male': 0, 'Female': 1},
    'Customer Type': {'Loyal Customer': 1, 'disloyal Customer': 0},
    'Type of Travel': {'Business travel': 1, 'Personal Travel': 0},
    'Class': {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
}

# Aplicar el mapeo
for col, mapping in categorical_mappings.items():
    X[col] = X[col].map(mapping)

# Las variables ordinales ya tienen valores numéricos entre 0 y 5, por lo que no necesitan codificación adicional
ordinal_cols = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 
                'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 
                'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

# Paso 3: Crear el preprocesador de columnas (solo escalar las numéricas)
numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols)  # Escalado para las columnas numéricas
    ]
)

# Paso 4: Crear los pipelines específicos de cada modelo
pipeline_rf = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_xgb = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss'))
])

pipeline_catboost = Pipeline(steps=[
    ('model', CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=False))
])

# Paso 5: Definir el modelo de Stacking
stacking_model = StackingClassifier(
    estimators=[
        ('catboost', pipeline_catboost),  # CatBoost
        ('xgboost', pipeline_xgb),        # XGBoost
        ('random_forest', pipeline_rf)    # Random Forest
    ],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42),
    cv=5
)

# Paso 6: Definir las métricas y la validación cruzada
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Paso 7: Función para evaluar un modelo
def evaluar_modelo_con_tiempo(pipeline, X, y, cv):
    start_time = time.time()  # Tiempo de inicio
    cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=False)
    end_time = time.time()  # Tiempo final
    elapsed_time = end_time - start_time  # Tiempo total de procesamiento
    
    # Convertir a formato legible de horas:minutos:segundos:milisegundos
    elapsed_time_readable = str(timedelta(seconds=elapsed_time))
    
    return cv_results, elapsed_time, elapsed_time_readable

# Paso 8: Definir el KFold para la validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lista de resultados
resultados = []

# Evaluar los modelos base y el modelo de Stacking
modelos = {
    'Random Forest': pipeline_rf,
    'XGBoost': pipeline_xgb,
    'CatBoost': pipeline_catboost,
    'Stacking': stacking_model
}

# Paso 9: Evaluar cada modelo con validación cruzada y almacenar métricas y tiempos
for nombre_modelo, modelo in modelos.items():
    print(f"\nEvaluando {nombre_modelo}...")
    
    # Evaluar el modelo y calcular métricas y tiempo
    cv_resultados, tiempo, tiempo_legible = evaluar_modelo_con_tiempo(modelo, X, y, kf)
    
    # Almacenar resultados en un diccionario
    resultados.append({
        'modelo': nombre_modelo,
        'accuracy_mean': cv_resultados['test_accuracy'].mean(),
        'precision_mean': cv_resultados['test_precision'].mean(),
        'recall_mean': cv_resultados['test_recall'].mean(),
        'f1_mean': cv_resultados['test_f1'].mean(),
        'roc_auc_mean': cv_resultados['test_roc_auc'].mean(),
        'tiempo_procesamiento': tiempo,
        'tiempo_procesamiento_legible': tiempo_legible,
        'modelo_pipeline': modelo  # Guardar el pipeline del modelo
    })

# Convertimos los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)

# Paso 10: Crear una vista truncada para mostrar en la consola (sin los pipelines)
df_vista_consola = df_resultados.drop(columns=['modelo_pipeline'])

# Mostrar los resultados sin la columna `modelo_pipeline` en la consola
print("\nResultados finales:")
print(df_vista_consola)

# Paso 11: Guardar el DataFrame completo (incluyendo pipelines) en un CSV
output_csv = 'resultados_modelos_completos.csv'
df_resultados.to_csv(output_csv, index=False)
print(f"\nLos resultados completos se han guardado en el archivo: {output_csv}")

# Paso 12: Entrenar el mejor modelo con todos los datos y guardarlo
mejor_modelo = df_resultados.loc[df_resultados['accuracy_mean'].idxmax()]

print(f"\nEntrenando el mejor modelo: {mejor_modelo['modelo']} con todos los datos...")

# Aquí entrenamos el mejor modelo en todos los datos
modelo_final = mejor_modelo['modelo_pipeline']
modelo_final.fit(X, y)

# Paso 13: Guardar el mejor modelo en un archivo con el nombre del algoritmo y la fecha/hora
def guardar_modelo_con_nombre(modelo, nombre_modelo):
    output_dir = 'C:/Users/erika/Documents/github/erika_alvares/G1_ML_Airline_Erika/data/modelos_entrenamiento/'
    os.makedirs(output_dir, exist_ok=True)
    
    # Obtener la fecha y hora actual para el nombre del archivo
    fecha_hora_actual = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Crear el nombre del archivo
    nombre_archivo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.pkl"
    
    # Guardar el modelo entrenado
    joblib.dump(modelo, os.path.join(output_dir, nombre_archivo))
    print(f"Modelo guardado exitosamente con el nombre: {nombre_archivo}")

# Guardar el modelo entrenado
guardar_modelo_con_nombre(modelo_final, mejor_modelo['modelo'])



Evaluando Random Forest...

Evaluando XGBoost...

Evaluando CatBoost...

Evaluando Stacking...

Resultados finales:
          modelo  accuracy_mean  precision_mean  recall_mean   f1_mean  roc_auc_mean  tiempo_procesamiento tiempo_procesamiento_legible
0  Random Forest       0.632373        0.583962     0.527764  0.554440      0.620075             94.664749               0:01:34.664749
1        XGBoost       0.677925        0.674597     0.496225  0.571813      0.656566              2.194517               0:00:02.194517
2       CatBoost       0.960191        0.968753     0.938415  0.953341      0.957631              9.353337               0:00:09.353337
3       Stacking       0.957044        0.962776     0.937123  0.949772      0.954702            532.321269               0:08:52.321269

Los resultados completos se han guardado en el archivo: resultados_modelos_completos.csv

Entrenando el mejor modelo: CatBoost con todos los datos...
Modelo guardado exitosamente con el nombre: CatBoost

In [15]:
# Librerías principales para manipulación de datos
import pandas as pd  # Manejo de datos en formato tabular (DataFrames)
import numpy as np  # Funciones matemáticas y operaciones con arrays

# Librerías para construir pipelines y preprocesamiento
from sklearn.pipeline import Pipeline  # Para crear pipelines que incluyan pasos de preprocesamiento y modelo
from sklearn.preprocessing import StandardScaler  # Para escalar variables numéricas
from sklearn.compose import ColumnTransformer  # Para aplicar diferentes transformaciones a diferentes tipos de columnas

# Modelos de machine learning
from sklearn.ensemble import RandomForestClassifier, StackingClassifier  # Algoritmo de clasificación basado en árboles y Stacking
from xgboost import XGBClassifier  # Algoritmo XGBoost, un modelo de boosting eficiente
from catboost import CatBoostClassifier  # Algoritmo CatBoost, un modelo de boosting que maneja variables categóricas internamente

# Librerías para validación cruzada y evaluación de modelos
from sklearn.model_selection import cross_validate, StratifiedKFold  # Validación cruzada y división estratificada de los datos
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score  # Métricas para evaluar el rendimiento de los modelos

# Librerías auxiliares para medir tiempos y guardar modelos
import time  # Para medir el tiempo de ejecución de los modelos
import os  # Para manejar el sistema de archivos, como la creación de directorios
import joblib  # Para guardar y cargar modelos entrenados
from datetime import datetime, timedelta  # Para obtener la fecha y hora actual

# Configuración cuaderno Jupyter: Mostrar todas las columnas y ajustar ancho de las columnas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Paso 1: Cargar los datos (CSV limpio)
data = pd.read_csv('../data/airline_passenger_satisfaction.csv')

# Paso 1.1: Eliminar registros con valores nulos en la columna 'Arrival Delay in Minutes'
data = data.dropna(subset=['Arrival Delay in Minutes'])

# Paso 1.2: Eliminar las columnas 'Unnamed: 0' y 'id' porque no aportan valor al análisis
data = data.drop(['Unnamed: 0', 'id'], axis=1)

# Paso 1.3: Convertir las etiquetas de 'satisfaction' de cadenas a valores numéricos (0 y 1)
data['satisfaction'] = data['satisfaction'].map({'satisfied': 1, 'neutral or dissatisfied': 0})
y = data['satisfaction']
X = data.drop('satisfaction', axis=1)

# Paso 2: Codificar manualmente las variables categóricas usando `map`
# Definir las columnas categóricas
categorical_cols = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

# Crear un mapeo para cada columna categórica
categorical_mappings = {
    'Gender': {'Male': 0, 'Female': 1},
    'Customer Type': {'Loyal Customer': 1, 'disloyal Customer': 0},
    'Type of Travel': {'Business travel': 1, 'Personal Travel': 0},
    'Class': {'Eco': 0, 'Eco Plus': 1, 'Business': 2}
}

# Aplicar el mapeo
for col, mapping in categorical_mappings.items():
    X[col] = X[col].map(mapping)

# Las variables ordinales ya tienen valores numéricos entre 0 y 5, por lo que no necesitan codificación adicional
ordinal_cols = ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 
                'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 
                'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']

# Paso 3: Crear el preprocesador de columnas (solo escalar las numéricas)
numerical_cols = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols)  # Escalado para las columnas numéricas
    ]
)

# Paso 4: Crear los pipelines específicos de cada modelo
pipeline_rf = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])

pipeline_xgb = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', XGBClassifier(n_estimators=100, random_state=42, eval_metric='logloss'))
])

pipeline_catboost = Pipeline(steps=[
    ('model', CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=False))
])

# Paso 5: Definir el modelo de Stacking
stacking_model = StackingClassifier(
    estimators=[
        ('catboost', pipeline_catboost),  # CatBoost
        ('xgboost', pipeline_xgb),        # XGBoost
        ('random_forest', pipeline_rf)    # Random Forest
    ],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42),
    cv=5
)

# Paso 6: Definir las métricas y la validación cruzada
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

# Paso 7: Función para evaluar un modelo
def evaluar_modelo_con_tiempo(pipeline, X, y, cv):
    start_time = time.time()  # Tiempo de inicio
    cv_results = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, return_train_score=False)
    end_time = time.time()  # Tiempo final
    elapsed_time = end_time - start_time  # Tiempo total de procesamiento
    
    # Convertir a formato legible de horas:minutos:segundos:milisegundos
    elapsed_time_readable = str(timedelta(seconds=elapsed_time))
    
    return cv_results, elapsed_time, elapsed_time_readable

# Paso 8: Definir el KFold para la validación cruzada
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Lista de resultados
resultados = []

# Evaluar los modelos base y el modelo de Stacking
modelos = {
    'Random Forest': pipeline_rf,
    'XGBoost': pipeline_xgb,
    'CatBoost': pipeline_catboost,
    'Stacking': stacking_model
}

# Paso 9: Evaluar cada modelo con validación cruzada y almacenar métricas y tiempos
for nombre_modelo, modelo in modelos.items():
    print(f"\nEvaluando {nombre_modelo}...")
    
    # Evaluar el modelo y calcular métricas y tiempo
    cv_resultados, tiempo, tiempo_legible = evaluar_modelo_con_tiempo(modelo, X, y, kf)
    
    # Almacenar resultados en un diccionario
    resultados.append({
        'modelo': nombre_modelo,
        'accuracy_mean': cv_resultados['test_accuracy'].mean(),
        'precision_mean': cv_resultados['test_precision'].mean(),
        'recall_mean': cv_resultados['test_recall'].mean(),
        'f1_mean': cv_resultados['test_f1'].mean(),
        'roc_auc_mean': cv_resultados['test_roc_auc'].mean(),
        'tiempo_procesamiento': tiempo,
        'tiempo_procesamiento_legible': tiempo_legible,
        'modelo_pipeline': modelo  # Guardar el pipeline del modelo
    })

# Convertimos los resultados en un DataFrame
df_resultados = pd.DataFrame(resultados)

# Paso 10: Crear una vista truncada para mostrar en la consola (sin los pipelines)
df_vista_consola = df_resultados.drop(columns=['modelo_pipeline'])

# Mostrar los resultados sin la columna `modelo_pipeline` en la consola
print("\nResultados finales:")
print(df_vista_consola)

# Paso 11: Guardar el DataFrame completo (incluyendo pipelines) en un CSV
output_csv = '../data/modelos_entrenamiento/resultados_modelos_completos.csv'
df_resultados.to_csv(output_csv, index=False)
print(f"\nLos resultados completos se han guardado en el archivo: {output_csv}")

# Paso 12: Entrenar el mejor modelo con todos los datos y guardarlo
mejor_modelo = df_resultados.loc[df_resultados['accuracy_mean'].idxmax()]

print(f"\nEntrenando el mejor modelo: {mejor_modelo['modelo']} con todos los datos...")

# Aquí entrenamos el mejor modelo en todos los datos
modelo_final = mejor_modelo['modelo_pipeline']
modelo_final.fit(X, y)

# Paso 13: Guardar el mejor modelo en un archivo con el nombre del algoritmo y la fecha/hora
def guardar_modelo_con_nombre(modelo, nombre_modelo):
    output_dir = '../data/modelos_entrenamiento/'
    os.makedirs(output_dir, exist_ok=True)
    
    # Obtener la fecha y hora actual para el nombre del archivo
    fecha_hora_actual = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    if nombre_modelo == 'CatBoost':
        nombre_archivo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.cbm"
        modelo.named_steps['model'].save_model(os.path.join(output_dir, nombre_archivo))
        print(f"Modelo CatBoost guardado exitosamente con el nombre: {nombre_archivo}")
    else:
        nombre_archivo = f"{nombre_modelo}_mejor_modelo_{fecha_hora_actual}.pkl"
        joblib.dump(modelo, os.path.join(output_dir, nombre_archivo))
        print(f"Modelo guardado exitosamente con el nombre: {nombre_archivo}")

# Guardar el modelo entrenado según el tipo de modelo
guardar_modelo_con_nombre(modelo_final, mejor_modelo['modelo'])



Evaluando Random Forest...

Evaluando XGBoost...

Evaluando CatBoost...

Evaluando Stacking...

Resultados finales:
          modelo  accuracy_mean  precision_mean  recall_mean   f1_mean  roc_auc_mean  tiempo_procesamiento tiempo_procesamiento_legible
0  Random Forest       0.632373        0.583962     0.527764  0.554440      0.620075             85.081491               0:01:25.081491
1        XGBoost       0.677925        0.674597     0.496225  0.571813      0.656566              1.898627               0:00:01.898627
2       CatBoost       0.960191        0.968753     0.938415  0.953341      0.957631              8.474000               0:00:08.474000
3       Stacking       0.957044        0.962776     0.937123  0.949772      0.954702            533.376614               0:08:53.376614

Los resultados completos se han guardado en el archivo: resultados_modelos_completos.csv

Entrenando el mejor modelo: CatBoost con todos los datos...
Modelo CatBoost guardado exitosamente con el nombre: